In [1]:
%matplotlib tk 
import mne
import numpy as np
import matplotlib.pyplot as plt
from kymatio.sklearn import Scattering2D
from kymatio.sklearn import Scattering1D
from kymatio.scattering2d.filter_bank import filter_bank
from kymatio.scattering2d.utils import fft2
from colorsys import hls_to_rgb

### Physionet

In [ ]:
epoch_file = '../preproc/Physionet_raw_epo.fif'
epochs = mne.read_epochs(epoch_file)
epochs = epochs.apply_baseline()
epochs.equalize_event_counts()

In [ ]:
tmin, tmax = -0.5, 2
epochs_data = epochs.get_data()
# labels = epochs.events[epochs.events[:,-1]!=3]
labels = epochs.events[:,-1]
labels = labels-2
# labels = epochs.events[:,-1]-2
print(f"Data Size: {epochs_data.shape}")

# train data 
epochs_train = epochs.copy().crop(tmin= tmin, tmax=tmax)
epochs_train_data = epochs_train.get_data()
# epochs_train_data = np.transpose(epochs_train_data, axes = (1, 2, 0)) # Sklearn data = n_sam * n_freq
print(f"Train Size: {epochs_train_data.shape}")

### BCI3 IVa

In [ ]:
# raw = extractBCI3(runs = 0, person_id = 3)
# raw = raw.filter(7,30,verbose = False).copy()

# event_data = mne.events_from_annotations(raw)
# event_marker, event_ids = event_data
# event_ids = dict(right = 1, foot = 3, test = 2)
# epochs = mne.Epochs(raw, event_marker, event_ids, tmin=-0.5, tmax=1, preload = True)

In [37]:
epoch_file = '../preproc/BCI3_car_[3]_P3_epo.fif'
epochs = mne.read_epochs(epoch_file)
event_ids = epochs.event_id

Reading ../preproc/BCI3_car_[3]_P3_epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 24) active
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
280 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [38]:
epochs = epochs.apply_proj().copy()
epochs = epochs.apply_baseline().copy()
epochs.equalize_event_counts()
tmin, tmax = -0.5, 2
# epochs.crop(tmin= tmin, tmax=tmax)
picks=['C1', 'C2','C3', 'C4', 'C5', 'C6', 'Cz']#,'T9','T10']
epochs.pick_channels(picks)
T1 = epochs['right']
T2 = epochs['foot']
t1 = epochs['right'].average()
t2 = epochs['foot'].average()
classes = list(event_ids.keys())
# mne.viz.plot_compare_evokeds([T1,-T2],picks= c);

Projections have already been applied. Setting proj attribute to True.
Applying baseline correction (mode: mean)
Dropped 202 epochs: 0, 1, 2, 3, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256

In [39]:
epochs_data = epochs.get_data()
# labels = epochs.events[epochs.events[:,-1]!=3]
# labels = epochs.events[:,-1]
# labels = labels-2
# labels = epochs.events[:,-1]-2
print(f"Data Size: {epochs_data.shape}")

# train data 
tmin, tmax = -0.5, 1
epochs_train = epochs.copy().crop(tmin= tmin, tmax=tmax)
epochs_train_data = epochs_train.get_data()
# epochs_train_data = np.transpose(epochs_train_data, axes = (1, 2, 0)) # Sklearn data = n_sam * n_freq
print(f"Train Size: {epochs_train_data.shape}")

Data Size: (78, 7, 151)
Train Size: (78, 7, 151)


In [40]:
epochs_data = epochs['right','foot'].get_data()
labels = np.concatenate([-np.ones(epochs['right'].get_data().shape[0]), 
                        np.ones(epochs['right'].get_data().shape[0])])
M,N = epochs_data.shape[1:]
J,L = 2,8
scat2D = Scattering2D(J=J,shape=(M,N),L=8)
epochs_data = epochs_data.reshape(epochs_data.shape[0],-1)

In [41]:
# Method1 [CH1,ch2,ch3]
# epochs_train_data = epochs_train_data.reshape((epochs_train_data.shape[0], -1))
# epochs_train_data.shape, labels.shape

# Method2 [T1, T2, T3 ]
# epochs_data = np.transpose(epochs_data,axes=(0,2,1))
# epochs_data = epochs_data.reshape((epochs_data.shape[0], -1))
epochs_data.shape, labels.shape


((52, 1057), (52,))

### Classification

In [33]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [42]:
clf = LDA()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(epochs_data, labels)
pipe = Pipeline([('scatter', scat2D), ('clf', clf)])
scores = cross_val_score(pipe, epochs_data, labels, cv = cv, verbose=False)

In [43]:
scores.mean()

0.509090909090909

In [ ]:
filters = filter_bank(68,121,4,10)
filters

# Scattering 1D Analysis

In [ ]:
start_time = 0
end_time = 100
time = np.arange(start_time, end_time, 1/100)
theta = 0
frequency = 5
amplitude = 1
y = np.concatenate((np.sin(2 * np.pi * frequency * time[0:5000] + theta), np.sin(2 * np.pi * 10 * time[5000:] + theta)))
# figure(figsize=(20, 6), dpi=80)
plt.plot(time,y)

T =  y.shape[-1]
J,L = 4,16
scat_1D = Scattering1D(J,T,L)
Sx1 = scat_1D(y)

plt.figure()
for i in range(len(scat_1D.psi1_f)):
    plt.plot((np.abs(scat_1D.psi1_f[i][0])))
plt.figure()
for i in range(len(scat_1D.psi2_f)):
    plt.plot((np.abs(scat_1D.psi2_f[i][0])))
plt.figure()
for i in range(len(scat_1D.phi_f)):
    plt.plot((scat_1D.phi_f[0]))

meta  = scat_1D.meta()
order0 = np.where(meta['order'] == 0)
order1 = np.where(meta['order'] == 1)
order2 = np.where(meta['order'] == 2)

In [ ]:
plt.figure()
for i in range(len(scat2D.psi[0])):
    plt.plot((scat2D.psi[i][0]))
plt.imshow((scat2D.psi[1][0]))